In [1]:

#pip install matplotlib

In [2]:
#imports
import pandas as pd
from qgis.core import *
import qgis.utils
import matplotlib.pyplot as plt
from qgis.analysis import QgsNativeAlgorithms
import processing
from processing.core.Processing import Processing

Application path not initialized


In [3]:
#setting up to run qgis code
# Supply path to qgis install location
QgsApplication.setPrefixPath("'/Applications/QGIS.app/Contents/MacOS'", True)

# Create a reference to the QgsApplication.  Setting the
# second argument to False disables the GUI.
qgs = QgsApplication([], False)

# Load providers
qgs.initQgis()

#getting functions
from qgis.core import *
import qgis.utils
import matplotlib.pyplot as plt
from qgis.analysis import QgsNativeAlgorithms
import processing
from processing.core.Processing import Processing
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())


Logged warning: Duplicate provider native registered


False

In [4]:
def suitability_df_formatting(file,j):
    df_suitability = pd.read_csv(file)
    df_suitability = df_suitability[['ADM2_ES','ADM2_PCODE', "_mean", "_median"]]#select columns
    df_suitability = df_suitability.rename(columns={'ADM2_PCODE':'MUN_CODE', 'ADM2_ES':'MUN', '_mean':'SUITABILITY_MEAN', '_median':'SUITABILITY_MEDIAN'})
    df_suitability['MUN_CODE'] = df_suitability['MUN_CODE'].str.replace('CO', "")
    df_suitability['MUN_CODE'] = df_suitability['MUN_CODE'].str.lstrip('0')
    df_suitability['MUN_CODE'] = df_suitability['MUN_CODE'].str.strip()
    if j == "":
        output_file = file[:-20] + "suitability_mean_median.csv"
    else:
        output_file = file[:-28] + "/suitability_mean_median.csv"
    print(output_file)
    df_suitability.to_csv(output_file, index = False)    

In [5]:
#calculatie zonal statistics, i.e. the mean, for the suitability
for i in ['245','585']:
    for j in ['', '_model1/']:
        if i == '585' and j == '':
            continue #as only need one historical file
        else:
            input_raster ='/Users/laurengomezcullen/maxent/colombia/results_historical_H/model1/' + i+'/Triatominae'+j[:-1]+'_avg.asc'
            output_raster = '/Users/laurengomezcullen/maxent/colombia/results_historical_H/'+ j[1:] +'/zonal'+j[:-1]+'_statistics.shp'
            sdm_mean_median= processing.run("native:zonalstatisticsfb", {'INPUT':'/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/borders/colombian_municipalities/col_admbnda_adm2_mgn_20200416.shp','INPUT_RASTER':input_raster,'RASTER_BAND':1,'COLUMN_PREFIX':'_','STATISTICS':[2,3],'OUTPUT':output_raster})

        
        # convert the shapefile with zonal statistics and save it as a csv file
        layer = QgsVectorLayer(output_raster, "sdm", "ogr")
        if j == "":
            output_csv = "/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/zonal"+j[:-1]+"_statistics"
        else:
            output_csv = "/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/"+i+"/"+j[1:-1]+"/zonal"+j[:-1]+"_statistics"
        QgsVectorFileWriter.writeAsVectorFormat(layer, output_csv,"utf-8",driverName = "CSV" , layerOptions = ['GEOMETRY=AS_XYZ'])
      
        
        # convert the csv to a suitability dataframe to use in the random forest
        file = output_csv + '.csv'
        print(file)
        suitability_df_formatting(file,j)

/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_30315/4104580605.py:18: DeprecationWarning: QgsVectorFileWriter.writeAsVectorFormat() is deprecated
  QgsVectorFileWriter.writeAsVectorFormat(layer, output_csv,"utf-8",driverName = "CSV" , layerOptions = ['GEOMETRY=AS_XYZ'])
Warning 1: Geometry type Multi Polygon is not compatible with GEOMETRY=AS_XYZ.


/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/zonal_statistics.csv
/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/suitability_mean_median.csv


/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_30315/4104580605.py:18: DeprecationWarning: QgsVectorFileWriter.writeAsVectorFormat() is deprecated
  QgsVectorFileWriter.writeAsVectorFormat(layer, output_csv,"utf-8",driverName = "CSV" , layerOptions = ['GEOMETRY=AS_XYZ'])
Warning 1: Geometry type Multi Polygon is not compatible with GEOMETRY=AS_XYZ.


/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/245/model1/zonal_model1_statistics.csv
/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/245/model1/suitability_mean_median.csv


/var/folders/x6/cq3jxj9j3zz2zgk2zvxn46q80000gn/T/ipykernel_30315/4104580605.py:18: DeprecationWarning: QgsVectorFileWriter.writeAsVectorFormat() is deprecated
  QgsVectorFileWriter.writeAsVectorFormat(layer, output_csv,"utf-8",driverName = "CSV" , layerOptions = ['GEOMETRY=AS_XYZ'])
Warning 1: Geometry type Multi Polygon is not compatible with GEOMETRY=AS_XYZ.


/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/585/model1/zonal_model1_statistics.csv
/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/585/model1/suitability_mean_median.csv


In [6]:

# for i in ['245','585']:
# file_path = '/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/245/model1/default_settings_zonal_statistics_testing.shp'
# layer = QgsVectorLayer(file_path, "sdm", "ogr")
# QgsVectorFileWriter.writeAsVectorFormat(layer, "/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/245/model1/default_settings_zonal_statistics_testing","utf-8",driverName = "CSV" , layerOptions = ['GEOMETRY=AS_XYZ'])

In [7]:
# df_suitability = pd.read_csv("/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/245/model1/default_settings_zonal_statistics_testing.csv")

# df_suitability = df_suitability[['ADM2_ES','ADM2_PCODE', "_mean"]]
# df_suitability = df_suitability.rename(columns={'ADM2_PCODE':'MUN_CODE', 'ADM2_ES':'MUN', '_mean':'SUITABILITY_MEAN'})
# df_suitability['MUN_CODE'] = df_suitability['MUN_CODE'].str.replace('CO', "")
# df_suitability['MUN_CODE'] = df_suitability['MUN_CODE'].str.lstrip('0')
# df_suitability['MUN_CODE'] = df_suitability['MUN_CODE'].str.strip()
# df_suitability.to_csv("/Users/laurengomezcullen/Documents/Cambridge/Fourth/Project/final/data/random_forest_input/colombia/245/model1/suitability_mean.csv", index = False)

In [8]:

# provider and layer registries from memory
qgs.exitQgis()